<a href="https://colab.research.google.com/github/xoXen/3113-Labs/blob/lab-notebooks/AY24_25_ICT3102_Week02_Introduction_to_Profiling_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.0 Learning Objectives

By the end of this lab, you would have learnt the following:

1.   How to use google colab
2.   The difference between deterministic profilers and statistical profilers.
3.   How to profile the **running time** of a piece of python code.
4.   How to profile the **memory consumption** of a piece of python code.
5.   How to profile the **cpu usage** of a piece of python code.

## 2.1.1 Google Colab

The Google colab is a free cloud service which allows us to execute code on Google's cloud servers. It is extremely convenient for us as it provides all of us with the same environment to work off from. Let's see what hardware Google is providing us with:

In [ ]:
from psutil import *
cpu_count()

Typically you will see the value of two here. As of now, google provides us with a dual core Intel Xeon either 2.20 or 2.30 GHz. You will be able to get more details of the software / hardware from the following:


## 2.2 Profiler types

In this section of the lab, we will be working in particular with profilers in the Python language. In general, there are at least two types of profilers in python. They are deterministic and statistical profiling. The differences are as follows:

**Deterministic profiling**

Deterministic profilers monitors all function calls, function returns and exception events. Timings are made for every single one of this events.

**Statistical profiling**

Instead of tracking every event (e.g. call to every function), statistical profilers interrupts the application periodically and samples the effective instruction pointer. These are analyzed next to estimate where the application is spending its time in.

*source: https://docs.python.org/2.0/lib/Deterministic_Profiling.html*

Typically deterministic profilers cause huge overheads as every single event is instrumented. However naturally it provides high granularity and accuracy of the data generated.

## 2.3 Profiling Time

We will be using two functions to measure time. The first is via the time library, using either the perf_counter module or just simply using the default_timer() function to get the current time. To illustrate this, let's use our favourite sorting algorithm, insertion sort.

In [ ]:
def insertion_sort(arr):
  for i in range(1, len(arr)):
    curr_val = arr[i]
    curr_index = i
    while curr_index > 0 and arr[curr_index-1] > curr_val:
      arr[curr_index] = arr[curr_index-1]
      curr_index -= 1

    arr[curr_index] = curr_val

In [ ]:
import random

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

### 2.3.1 time.perf_counter

Let's find out how long insertion sort takes on these numbers using time.perf_counter.

In [ ]:
from time import perf_counter



### 2.3.2 timeit

Another way (and some may argue a better way) is to use the timeit function. This allows us to measure how long a function takes to execute, when it is running many iterations.

In [ ]:
from timeit import timeit



Let's now compare insertion sort with merge sort.

In [ ]:
def merge_sort(array, left_index, right_index):
    if left_index >= right_index:
        return

    middle = (left_index + right_index)//2
    merge_sort(array, left_index, middle)
    merge_sort(array, middle + 1, right_index)
    merge(array, left_index, right_index, middle)

def merge(array, left_index, right_index, middle):
    # Make copies of both arrays we're trying to merge

    # The second parameter is non-inclusive, so we have to increase by 1
    left_copy = array[left_index:middle + 1]
    right_copy = array[middle+1:right_index+1]

    # Initial values for variables that we use to keep
    # track of where we are in each array
    left_copy_index = 0
    right_copy_index = 0
    sorted_index = left_index

    # Go through both copies until we run out of elements in one
    while left_copy_index < len(left_copy) and right_copy_index < len(right_copy):

        # If our left_copy has the smaller element, put it in the sorted
        # part and then move forward in left_copy (by increasing the pointer)
        if left_copy[left_copy_index] <= right_copy[right_copy_index]:
            array[sorted_index] = left_copy[left_copy_index]
            left_copy_index = left_copy_index + 1
        # Opposite from above
        else:
            array[sorted_index] = right_copy[right_copy_index]
            right_copy_index = right_copy_index + 1

        # Regardless of where we got our element from
        # move forward in the sorted part
        sorted_index = sorted_index + 1

    # We ran out of elements either in left_copy or right_copy
    # so we will go through the remaining elements and add them
    while left_copy_index < len(left_copy):
        array[sorted_index] = left_copy[left_copy_index]
        left_copy_index = left_copy_index + 1
        sorted_index = sorted_index + 1

    while right_copy_index < len(right_copy):
        array[sorted_index] = right_copy[right_copy_index]
        right_copy_index = right_copy_index + 1
        sorted_index = sorted_index + 1

# credits: https://stackabuse.com/merge-sort-in-python/

### 2.3.3 cProfile

Let's first look at a simple iterative example. Assume that we have 5 functions, and they are related to each other in the following manner:

```
func_a -> func_b (1000 times) -> func_e (17000 times)
       -> func_c (1000 times) -> func_e (15000 times)
```

In [ ]:
import timeit

tic = timeit.default_timer();

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,17000):
    func_e()

def func_c():
  for i in range(0,15000):
    func_e()

def func_e():
  return

func_a()
toc = timeit.default_timer();
print(f"{toc-tic}")

If we were to run the code above, the time taken is within the range of 2 to 3 seconds. Let's now try to profile the code with the cProfile module. Using the magic %% operator, we are able to write the cell contents to the file in our google cloud server.

In [ ]:
%%writefile multiple_calls.py

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,17000):
    func_e()

def func_c():
  for i in range(0,15000):
    func_e()

def func_e():
  return

func_a()

And now the functions can be profiled with the following line:

We can sort it with the -s option.

Notice that the execution time increases almost three times. But here you can see clearly the various calls being made, total time being spent as well as the cummulative time. The column headings are:

**ncalls**
Total number of calls.

**tottime**
Total time spent in the given function (and excluding time made in calls to sub-functions).

**percall**
tottime divided by ncalls

**cumtime**
Cumulative time spent in this and all subfunctions (from invocation till exit).

**percall**
cumtime divided by primitive calls

**filename:lineno(function)**
Line number of the function

*Additional options can be seen here: https://docs.python.org/3.7/library/profile.html*


Let's try to visualize these calls. We first install the necessary packages:

Next we generate the output files in the following manner. Notice that the files output.png and output.pstats are generated on the folder structure on the left.

In [ ]:
!python -m cProfile -o output.pstats multiple_calls.py
!gprof2dot -f pstats output.pstats | dot -Tpng -o output.png

The following line displays the output image on your notebook. You can see that it corresponds to the original call tree up above.

Now let's try to carry out this analysis on our favourite fibonacci number recursive function, and see if there is a way to optimize it. First we find out what the execution time is when n is equals to 10.

Let's try to do the profiling here from a module. For this recursive function, it's clearer to use the cProfile as a module rather than from the command line.

Notice that the execution time has almost doubled with such a small example. You can also tell that the recursive call occurs for more than 18454929 times, although the number of primitive calls are just 40. Hence you can deduce that there is room for improvement here. Let's try to carry out some memoization to reduce the number of function calls.

We do that by first creating a dictionary and storing the outputs there. If the output is present, we will use the value in the dictionary. Else we will carry out the function call.

In [ ]:
import cProfile
import timeit

tic = timeit.default_timer();

fib_num = 34

def fib(fib_dict, n):
  if n in fib_dict: return fib_dict[n]

  if n == 0: return 0
  if n == 1: return 1

  if (n-1) in fib_dict:
    return_1 = fib_dict.get(n-1)
  else:
    return_1 = fib(fib_dict, n-1)
    fib_dict[n-1] = return_1

  if (n-2) in fib_dict:
    return_2 = fib_dict.get(n-2)
  else:
    return_2 = fib(fib_dict, n-1)
    fib_dict[n-2] = return_2

  fib_dict[n] = fib_dict[n-1] + fib_dict[n-2]
  return fib_dict[n]

def fib_driver(n):
  fib_dict = {}
  fib(fib_dict, n)
  print(fib_dict[n-1])

cProfile.run('print(f\"{fib_driver(fib_num)} {timeit.default_timer() - tic}\")')

You would notice that the number of iterative calls have decreased dramatically to just 35, which in fact makes this recursive function O(n), the same as the iterative version of our fibonacci number generator.

As you can see, it is in the same order as that of the iterative version we have been working with so far.

### 2.3.4 py-spy

We will now compare cProfile with a statistical profiler, py-spy.

Let's first create a python script which runs for at least 10 seconds.

In [ ]:
%%writefile long_multiple_calls.py

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,77000):
    func_e()

def func_c():
  for i in range(0,55000):
    func_e()

def func_e():
  return

func_a()

What the next line does is to run the function, sampling it at various intervals. In this instance, it is sampled at 100 times per second.

What we see on top is known as a flame graph. We can discuss a bit more on this flame graph.

The flame graph is a collection of stack traces and these traces are arranged as column of boxes, where each box represents a function or a stack frame.

The y-axis shows the depth and in this case, it is ordered from the top (root) to the bottom (leaf).

The x-axis spans the stack trace collection and the ordering is in alphabetical, and not by time.

The width of each function box shows the frequency at which that function was present in the stack traces. So in this example, it shows clearly that the bulk of the cummulative time was spent in calling function_b and function_c from function_a.

The background colors for each box are chosen purely for asthestic purposes and have no inherent meaning.

*source: https://cacm.acm.org/magazines/2016/6/202665-the-flame-graph/fulltext*

One of the useful commands is the Top command. Top shows a live view of what functions are taking the most time in your python program. Running py-spy with the top command gives you the following live statistical updates:

Let us compare this with cProfile to see if there is any delta between the two.

### 2.4 Profiling Memory

We will be showcasing three tools to profile memory. All three tools have their different strengths / weaknesses, and the choice of the tool is dependent on the situation.

### 2.4.1 Memory Profiler

In [ ]:
%%writefile insert_sort.py

import random
from memory_profiler import profile

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

@profile
def insertion_sort(arr):
  for i in range(1, len(arr)):
    curr_val = arr[i]
    curr_index = i
    while curr_index > 0 and arr[curr_index-1] > curr_val:
      arr[curr_index] = arr[curr_index-1]
      curr_index -= 1

    arr[curr_index] = curr_val

init_random_array(1000)
insertion_sort(array)

In [ ]:
%%writefile merge_sort.py

import random
from memory_profiler import profile

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

@profile
def merge_sort(array, left_index, right_index):
    if left_index >= right_index:
        return

    middle = (left_index + right_index)//2
    merge_sort(array, left_index, middle)
    merge_sort(array, middle + 1, right_index)
    merge(array, left_index, right_index, middle)

def merge(array, left_index, right_index, middle):
    # Make copies of both arrays we're trying to merge

    # The second parameter is non-inclusive, so we have to increase by 1
    left_copy = array[left_index:middle + 1]
    right_copy = array[middle+1:right_index+1]

    # Initial values for variables that we use to keep
    # track of where we are in each array
    left_copy_index = 0
    right_copy_index = 0
    sorted_index = left_index

    # Go through both copies until we run out of elements in one
    while left_copy_index < len(left_copy) and right_copy_index < len(right_copy):

        # If our left_copy has the smaller element, put it in the sorted
        # part and then move forward in left_copy (by increasing the pointer)
        if left_copy[left_copy_index] <= right_copy[right_copy_index]:
            array[sorted_index] = left_copy[left_copy_index]
            left_copy_index = left_copy_index + 1
        # Opposite from above
        else:
            array[sorted_index] = right_copy[right_copy_index]
            right_copy_index = right_copy_index + 1

        # Regardless of where we got our element from
        # move forward in the sorted part
        sorted_index = sorted_index + 1

    # We ran out of elements either in left_copy or right_copy
    # so we will go through the remaining elements and add them
    while left_copy_index < len(left_copy):
        array[sorted_index] = left_copy[left_copy_index]
        left_copy_index = left_copy_index + 1
        sorted_index = sorted_index + 1

    while right_copy_index < len(right_copy):
        array[sorted_index] = right_copy[right_copy_index]
        right_copy_index = right_copy_index + 1
        sorted_index = sorted_index + 1

# credits: https://stackabuse.com/merge-sort-in-python/

init_random_array(10)
merge_sort(array, 0, len(array)-1)

Notice that the logs are really hard to read. Primarily it is because of the recursive call, hence for the same function gets called recursively many times. Each time it is called it is difficult to see the memory allocated. You may need to run these more than once, obtain the average to get a good stable result.

### 2.4.2 tracemalloc

In [ ]:
def partition(array, start, end):
    pivot = array[start]
    low = start + 1
    high = end

    while True:
        # If the current value we're looking at is larger than the pivot
        # it's in the right place (right side of pivot) and we can move left,
        # to the next element.
        # We also need to make sure we haven't surpassed the low pointer, since that
        # indicates we have already moved all the elements to their correct side of the pivot
        while low <= high and array[high] >= pivot:
            high = high - 1

        # Opposite process of the one above
        while low <= high and array[low] <= pivot:
            low = low + 1

        # We either found a value for both high and low that is out of order
        # or low is higher than high, in which case we exit the loop
        if low <= high:
            array[low], array[high] = array[high], array[low]
            # The loop continues
        else:
            # We exit out of the loop
            break

    array[start], array[high] = array[high], array[start]

    return high

def quick_sort(array, start, end):
    if start >= end:
        return

    p = partition(array, start, end)
    quick_sort(array, start, p-1)
    quick_sort(array, p+1, end)